In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# 下載套件

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision.ops import nms
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import numpy as np
import os
from pycocotools.coco import COCO
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pad_sequence
from pycocotools.cocoeval import COCOeval
import json
import tempfile
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import Adam
import random
import time
from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/data

/content/drive/MyDrive/Colab Notebooks/data


# dataoader

In [ ]:
class VOCDataset(Dataset):
    def __init__(self, img_dir, mask_dir, img_size=(640, 640)):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
        self.img_size = img_size

        # 建立顏色對應表
        self.color_to_class = self._build_color_mapping()

        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def _build_color_mapping(self):
        print("建立顏色對應表...")
        unique_colors = set()

        for img_file in self.img_files:
            mask_path = os.path.join(self.mask_dir, img_file.replace('.jpg', '.png'))
            if os.path.exists(mask_path):
                mask = Image.open(mask_path).convert('RGB')
                mask_array = np.array(mask)
                colors = mask_array.reshape(-1, 3)
                unique_colors.update([tuple(color) for color in colors])

        # 將黑白加入到21類
        bg_colors = {(0, 0, 0), (255, 255, 255)}

        # 移除黑白色，單獨處理
        color_list = sorted(list(unique_colors - bg_colors))
        color_to_class = {}

        # 先將黑白映射到 21
        color_to_class[(0, 0, 0)] = 21
        color_to_class[(255, 255, 255)] = 21

        # 其他最多取前20個顏色
        for i, color in enumerate(color_list):
            if i < 20:
                color_to_class[color] = i
            else:
                color_to_class[color] = 21  # 多餘顏色也映射為背景類

        print(f"找到 {len(unique_colors)} 種顏色，映射為 0~19 有效類別 + 背景類")
        print("顏色對應表:")
        for color, class_id in list(color_to_class.items()):
            print(f"  RGB{color} -> 類別{class_id}")

        return color_to_class


    def _rgb_to_class(self, mask_array):
        """將RGB mask轉換為類別索引"""
        h, w, c = mask_array.shape
        class_mask = np.zeros((h, w), dtype=np.long)

        # 為每個像素找到對應的類別
        for color, class_id in self.color_to_class.items():
            # 找到匹配這個顏色的所有像素
            matches = np.all(mask_array == color, axis=2)
            class_mask[matches] = class_id

        return class_mask

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png'))

        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        mask = Image.open(mask_path).convert('RGB')
        mask = mask.resize(self.img_size, Image.NEAREST)
        mask_array = np.array(mask)

        class_mask = self._rgb_to_class(mask_array)
        mask_tensor = torch.from_numpy(class_mask).long()

        return img, mask_tensor

# class數據集：miniimage160（10 類）
class Image160Dataset(Dataset):
    def __init__(self, root_dir, img_size=(640, 640)):
        self.root_dir = root_dir
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.img_files = []
        self.labels = []
        for idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls)
            files = [f for f in os.listdir(cls_dir) if f.endswith('.JPEG')]
            self.img_files.extend([os.path.join(cls, f) for f in files])
            self.labels.extend([idx] * len(files))
        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_files[idx])
        label = self.labels[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

# 自定義 collate_fn 處理變長張量
def collate_fn(batch):
    # 將圖像堆疊，圖像形狀應一致
    images = torch.stack([item[0] for item in batch])

    boxes = [item[1]['boxes'] for item in batch]
    labels = [item[1]['labels'] for item in batch]
    conf = [item[1]['scores'].unsqueeze(1) for item in batch]
    image_ids = [item[1]['image_id'] for item in batch]
    areas = [item[1]['area'] for item in batch]
    iscrowds = [item[1]['iscrowd'] for item in batch]

    # 讓 boxes, labels, conf 形狀一致
    boxes_padded = pad_sequence(boxes, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
    conf_padded = pad_sequence(conf, batch_first=True, padding_value=0)
    areas_padded = pad_sequence(areas, batch_first=True, padding_value=0)
    iscrowds_padded = pad_sequence(iscrowds, batch_first=True, padding_value=0)

    targets = {
        'boxes': boxes_padded,
        'labels': labels_padded,
        'scores': conf_padded,
        'image_id': torch.stack(image_ids),  # Stack image_ids
        'area': areas_padded,
        'iscrowd': iscrowds_padded
    }
    return images, targets

# det數據集：mini-coco_det
class COCODataset(Dataset):
    def __init__(self, data_root, split='train2017', img_size=(640, 640)):
        self.data_root = data_root
        self.img_dir = os.path.join(data_root, split)
        self.ann_file = os.path.join(data_root, 'annotations', f'instances_{split}.json')
        self.coco = COCO(self.ann_file)
        self.img_ids = list(self.coco.imgs.keys())
        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_dir, img_info['file_name'])
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        boxes = torch.tensor([[ann['bbox'][0], ann['bbox'][1],
                               ann['bbox'][0] + ann['bbox'][2], ann['bbox'][1] + ann['bbox'][3]]
                              for ann in anns], dtype=torch.float32)
        # Adjust category_id from 1-based (1-10) to 0-based (0-9)
        labels = torch.tensor([ann['category_id'] - 1 for ann in anns], dtype=torch.long)
        conf = torch.ones(len(anns))
        areas = torch.tensor([ann['area'] for ann in anns], dtype=torch.float32)
        iscrowds = torch.tensor([ann['iscrowd'] for ann in anns], dtype=torch.long)

        return img, {
            'boxes': boxes,
            'labels': labels,
            'scores': conf,
            'image_id': torch.tensor([img_id], dtype=torch.long),
            'area': areas,
            'iscrowd': iscrowds
        }


# 後處理det的輸出

In [ ]:
def post_process_det(det, stride=8, n_anchors=3, c_det=10, img_size=640):
    batch_size, _, h, w = det.shape
    h = h.item() if isinstance(h, torch.Tensor) else h
    w = w.item() if isinstance(w, torch.Tensor) else w

    det = det.reshape(batch_size, n_anchors, 5 + c_det, h, w)
    det = det.permute(0, 1, 3, 4, 2)  # [B, A, H, W, 5+C]
    # print(det[..., 0])
    # 修正座標解碼
    grid_x = torch.arange(w, device=det.device, dtype=torch.float32).reshape(1, 1, 1, w).expand(batch_size, n_anchors, h, w)
    grid_y = torch.arange(h, device=det.device, dtype=torch.float32).reshape(1, 1, h, 1).expand(batch_size, n_anchors, h, w)
    # print(grid_x)
    # 使用 sigmoid 來限制座標範圍
    cx = (torch.sigmoid(det[..., 0]) + grid_x) * stride
    cy = (torch.sigmoid(det[..., 1]) + grid_y) * stride
    w_pred = torch.exp(det[..., 2]) * stride
    h_pred = torch.exp(det[..., 3]) * stride

    x1 = cx - w_pred / 2
    y1 = cy - h_pred / 2
    x2 = cx + w_pred / 2
    y2 = cy + h_pred / 2

    # 確保座標在合理範圍內
    x1 = torch.clamp(x1, 0, img_size)
    y1 = torch.clamp(y1, 0, img_size)
    x2 = torch.clamp(x2, 0, img_size)
    y2 = torch.clamp(y2, 0, img_size)

    boxes = torch.stack([x1, y1, x2, y2], dim=-1)
    boxes = boxes.reshape(batch_size, n_anchors * h * w, 4)

    conf = torch.sigmoid(det[..., 4]).reshape(batch_size, n_anchors * h * w)
    cls_probs = torch.softmax(det[..., 5:], dim=-1).reshape(batch_size, n_anchors * h * w, c_det)

    outputs = []
    for i in range(batch_size):
        # 過濾低信心度的預測
        conf_mask = conf[i] > 0.5
        if conf_mask.sum() == 0:
            outputs.append({
                'boxes': torch.empty(0, 4),
                'labels': torch.empty(0, dtype=torch.long),
                'scores': torch.empty(0)
            })
            continue

        filtered_boxes = boxes[i][conf_mask]
        filtered_conf = conf[i][conf_mask]
        filtered_cls = cls_probs[i][conf_mask]

        # 最終分數
        max_cls_probs, predicted_labels = torch.max(filtered_cls, dim=-1)
        final_scores = filtered_conf * max_cls_probs

        # NMS
        keep_indices = nms(filtered_boxes, final_scores, iou_threshold=0.6)

        outputs.append({
            'boxes': filtered_boxes[keep_indices],
            'labels': predicted_labels[keep_indices],
            'scores': final_scores[keep_indices]
        })

    return outputs

# 資源計算

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Loss與評估指標

In [ ]:
# IoU 損失，拿來計算bbox的重疊
class CIoULoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred_boxes, target_boxes):
        iou = self.compute_iou(pred_boxes, target_boxes)
        ciou = iou
        return 1 - ciou.mean()

    def compute_iou(self, box1, box2):
        x1 = torch.max(box1[:, 0], box2[:, 0])
        y1 = torch.max(box1[:, 1], box2[:, 1])
        x2 = torch.min(box1[:, 2], box2[:, 2])
        y2 = torch.min(box1[:, 3], box2[:, 3])
        inter = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
        area1 = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
        area2 = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])
        union = area1 + area2 - inter
        return inter / (union + 1e-6)

##############################################
class DetectionLoss(nn.Module):
    def __init__(self, n_anchors=3, c_det=10, stride=8, anchor_sizes=None):
        super().__init__()
        self.n_anchors = n_anchors
        self.c_det = c_det
        self.stride = stride
        self.ciou_loss_fn = CIoULoss()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.cls_loss = nn.CrossEntropyLoss()
        # anchor尺寸
        if anchor_sizes is None:
            self.anchor_sizes = torch.tensor([
                [10, 13], [16, 30], [33, 23],
                # 可以根據數據集調整這些anchor尺寸
            ], dtype=torch.float32)
        else:
            self.anchor_sizes = torch.tensor(anchor_sizes, dtype=torch.float32)
        # 設定各loss的權重
        self.lambda_coord = 5.0
        self.lambda_obj = 1.0
        self.lambda_noobj = 0.5
        self.lambda_class = 1.0

    def forward(self, det_pred, targets):
        batch_size, _, h, w = det_pred.shape
        det_pred_device = det_pred.device
        if self.anchor_sizes.device != det_pred_device:
            self.anchor_sizes = self.anchor_sizes.to(det_pred_device)

        # 將預測reshape
        det_pred = det_pred.view(batch_size, self.n_anchors, 5 + self.c_det, h, w)
        det_pred = det_pred.permute(0, 1, 3, 4, 2)  # [B, A, H, W, 5+C]

        total_loss = 0
        num_pos = 0

        for b in range(batch_size):
            tgt_boxes = targets['boxes'][b]
            tgt_labels = targets['labels'][b]

            valid_mask = (tgt_labels >= 0) & (tgt_boxes.sum(dim=1) > 0)
            tgt_boxes = tgt_boxes[valid_mask]
            tgt_labels = tgt_labels[valid_mask]

            # 代表我的資料裡沒有物件
            if tgt_boxes.size(0) == 0:
                obj_pred = det_pred[b, :, :, :, 4].reshape(-1)
                obj_target = torch.zeros_like(obj_pred)
                total_loss += self.lambda_noobj * self.bce_loss(obj_pred, obj_target)
                continue

            assignments = self.assign_targets(det_pred[b], tgt_boxes, h, w)

            if len(assignments) == 0:
                obj_pred = det_pred[b, :, :, :, 4].reshape(-1)
                obj_target = torch.zeros_like(obj_pred)
                total_loss += self.lambda_noobj * self.bce_loss(obj_pred, obj_target)
                continue

            pred_boxes_list = []
            pred_obj_list = []
            pred_cls_list = []
            matched_tgt_boxes = []
            matched_tgt_labels = []

            for assignment in assignments:
                anchor_idx, grid_y, grid_x, tgt_idx = assignment
                #將預測框轉為座標
                pred_box = self.decode_single_prediction(
                    det_pred[b, anchor_idx, grid_y, grid_x, :4],
                    grid_y, grid_x, h, w
                )
                pred_boxes_list.append(pred_box)

                # pred信心度與class
                pred_obj_list.append(det_pred[b, anchor_idx, grid_y, grid_x, 4])
                pred_cls_list.append(det_pred[b, anchor_idx, grid_y, grid_x, 5:])

                # gt
                matched_tgt_boxes.append(tgt_boxes[tgt_idx])
                matched_tgt_labels.append(tgt_labels[tgt_idx])

            if len(pred_boxes_list) == 0:
                continue

            pred_boxes = torch.stack(pred_boxes_list)
            pred_obj = torch.stack(pred_obj_list)
            pred_cls = torch.stack(pred_cls_list)
            matched_tgt_boxes = torch.stack(matched_tgt_boxes)
            matched_tgt_labels = torch.stack(matched_tgt_labels)

            # 個個loss
            # 1. 座標損失
            coord_loss = self.ciou_loss_fn(pred_boxes, matched_tgt_boxes)

            # 2. 信心度損失 (正樣本)
            obj_target = torch.ones_like(pred_obj)
            obj_loss_pos = self.bce_loss(pred_obj, obj_target)

            # 3. class損失
            cls_loss = self.cls_loss(pred_cls, matched_tgt_labels)

            # 4. 信心度損失 (負樣本) - 所有位置
            obj_pred_all = det_pred[b, :, :, :, 4].reshape(-1)
            obj_target_all = torch.zeros_like(obj_pred_all)

            # 將正樣本的位置設1
            for assignment in assignments:
                anchor_idx, grid_y, grid_x, _ = assignment
                flat_idx = anchor_idx * h * w + grid_y * w + grid_x
                obj_target_all[flat_idx] = 1.0

            obj_loss_all = self.bce_loss(obj_pred_all, obj_target_all)

            total_loss += (
                self.lambda_coord * coord_loss +
                self.lambda_obj * obj_loss_pos +
                self.lambda_noobj * obj_loss_all +
                self.lambda_class * cls_loss
            )

            num_pos += len(assignments)

        return total_loss / max(num_pos, 1)

    def compute_anchor_iou(self, gt_wh, anchor_wh):
        # gt_wh: [N, 2] (width, height)
        # anchor_wh: [A, 2] (width, height)

        gt_wh = gt_wh.unsqueeze(1)  # [N, 1, 2]
        anchor_wh = anchor_wh.unsqueeze(0)  # [1, A, 2]

        # 計算交集面積 (假设中心對齊)
        inter_wh = torch.min(gt_wh, anchor_wh)  # [N, A, 2]
        inter_area = inter_wh[:, :, 0] * inter_wh[:, :, 1]  # [N, A]

        # 計算各自面積
        gt_area = gt_wh[:, :, 0] * gt_wh[:, :, 1]  # [N, 1]
        anchor_area = anchor_wh[:, :, 0] * anchor_wh[:, :, 1]  # [1, A]

        # 計算聯集面積
        union_area = gt_area + anchor_area - inter_area  # [N, A]

        # 計算IoU
        iou = inter_area / (union_area + 1e-6)  # [N, A]

        return iou

    def assign_targets(self, pred, tgt_boxes, h, w):
        assignments = []

        # 計算所有目標的寬高
        tgt_wh_list = []
        for tgt_box in tgt_boxes:
            width = tgt_box[2] - tgt_box[0]
            height = tgt_box[3] - tgt_box[1]
            tgt_wh_list.append([width, height])

        if len(tgt_wh_list) == 0:
            return assignments

        tgt_wh = torch.tensor(tgt_wh_list, device=tgt_boxes.device)  # [N, 2]

        # 計算anchor IoU
        anchor_ious = self.compute_anchor_iou(tgt_wh, self.anchor_sizes[:self.n_anchors])  # [N, A]

        for i, tgt_box in enumerate(tgt_boxes):
            # 計算目標中心點在網格中的位置
            cx = (tgt_box[0] + tgt_box[2]) / 2
            cy = (tgt_box[1] + tgt_box[3]) / 2

            grid_x = int(cx / self.stride)
            grid_y = int(cy / self.stride)

            # 確保在網格範圍內
            grid_x = max(0, min(w-1, grid_x))
            grid_y = max(0, min(h-1, grid_y))

            # 選擇IoU最高的anchor
            best_anchor = torch.argmax(anchor_ious[i]).item()

            assignments.append((best_anchor, grid_y, grid_x, i))

        return assignments

    def decode_single_prediction(self, pred_box, grid_y, grid_x, h, w):
        # 將匡轉為座標
        cx = (torch.sigmoid(pred_box[0]) + grid_x) * self.stride
        cy = (torch.sigmoid(pred_box[1]) + grid_y) * self.stride
        w_pred = torch.exp(pred_box[2]) * self.stride
        h_pred = torch.exp(pred_box[3]) * self.stride

        x1 = cx - w_pred / 2
        y1 = cy - h_pred / 2
        x2 = cx + w_pred / 2
        y2 = cy + h_pred / 2

        return torch.stack([x1, y1, x2, y2])


# mIOU
def compute_iou(preds, targets, num_classes):
    ious = []
    preds = preds.view(-1)
    targets = targets.view(-1)

    for cls in range(num_classes):
        pred_inds = preds == cls
        target_inds = targets == cls
        intersection = (pred_inds & target_inds).sum().item()
        union = (pred_inds | target_inds).sum().item()

        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(intersection / union)
    return np.nanmean(ious)

# mAP
def evaluate_map(model, dataloader, coco_gt, iou_type='bbox'):
    model.eval()
    results = []
    img_ids = []

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Evaluating"):
            images = images.to(device)
            outputs = post_process_det(model(images)[0], stride=8, n_anchors=3, c_det=10)

            for i, output in enumerate(outputs):
                image_id = int(targets['image_id'][i].item())
                img_ids.append(image_id)

                boxes = output['boxes'].cpu().numpy()
                scores = output['scores'].cpu().numpy()
                labels = output['labels'].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    if score > 0.2:  # 篩掉一些分數較低的框
                        x1, y1, x2, y2 = box
                        width = float(x2 - x1)
                        height = float(y2 - y1)

                        if width > 0 and height > 0: #發現我的眶有的x1會>x2，所以在這把她刪除
                            result = {
                                'image_id': int(image_id),
                                'category_id': int(label) + 1,
                                'bbox': [float(x1), float(y1), float(width), float(height)],
                                'score': float(score),
                            }
                            results.append(result)

    print(f"Total detections: {len(results)}")

    if not results:
        print("警告：沒有有效的檢測結果！")
        return 0.0
    with tempfile.NamedTemporaryFile(mode='w+', suffix='.json', delete=False) as f:
        json.dump(results, f)
        f.flush()
        coco_dt = coco_gt.loadRes(f.name)

    coco_eval = COCOeval(coco_gt, coco_dt, iouType=iou_type)
    coco_eval.params.imgIds = img_ids
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[0]


# model自定義

In [ ]:
# model
# 使用YOLOv8n當作我的backbone
class YOLOv8nBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = YOLO('yolov8n.pt').model.to(device)
        self.backbone = nn.Sequential(*list(base_model.model.children())[:10])

    def forward(self, x):
        outputs = []
        for i, layer in enumerate(self.backbone):
            x = layer(x)
            if i in [4, 6, 9]:
                outputs.append(x)
        return outputs

# FPN 我的neck
class FPN(nn.Module):
    def __init__(self, in_channels_list=[256, 512, 576], out_channels=512):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels_list[2], out_channels, 1)
        self.conv2 = nn.Conv2d(in_channels_list[1], out_channels, 1)
        self.conv3 = nn.Conv2d(in_channels_list[0], out_channels, 1)
        self.conv_fuse = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        p4, p6, p9 = x
        p9 = self.conv1(p9)
        p9 = F.interpolate(p9, scale_factor=2, mode='bilinear')
        p6 = self.conv2(p6)
        p4 = self.conv3(p4)
        p4 = F.interpolate(p4, scale_factor=0.5, mode='bilinear')
        fused = p9 + p6 + p4
        return self.relu(self.bn(self.conv_fuse(fused)))
# 定義我的head
class MultiTaskHead(nn.Module):
    def __init__(self, in_channels, c_det=10, c_seg=20, c_cls=10, n_anchors=3, img_size=640):
        super().__init__()
        self.n_anchors = n_anchors
        self.c_det = c_det
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(128, n_anchors * (5 + c_det) + c_seg, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.cls_fc = nn.Linear(128, c_cls)
        self.upsample = nn.Upsample(size=(img_size, img_size), mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        det_seg = self.conv2(x)
        det = det_seg[:, :self.n_anchors * (5 + self.c_det), :, :]
        seg = det_seg[:, self.n_anchors * (5 + self.c_det):, :, :]
        seg = self.upsample(seg)
        cls = self.cls_fc(self.pool(x).flatten(1))
        return det, seg, cls

class MultiTaskModel(nn.Module):
    def __init__(self, img_size=640):
        super().__init__()
        self.backbone = YOLOv8nBackbone()
        self.neck = FPN(in_channels_list=[64, 128, 256], out_channels=256)
        self.head = MultiTaskHead(in_channels=256, c_det=10, c_seg=20, c_cls=10, n_anchors=3, img_size=img_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        features = self.backbone(x)
        fused = self.neck(features)
        det, seg, cls = self.head(fused)
        return det, seg, cls

    def compute_iou(self, preds, targets, num_classes):
        iou_sum = 0
        for c in range(num_classes):
            pred_mask = (preds == c)
            target_mask = (targets == c)
            intersection = (pred_mask & target_mask).sum().float()
            union = (pred_mask | target_mask).sum().float()
            iou_sum += (intersection + 1e-6) / (union + 1e-6)
        return iou_sum / num_classes
    def train_stage(self, stage, train_loader, val_loader, criterion, evaluate_map=None, num_epochs=5, prev_ewc=None, save_path=None):
        optimizer = Adam(self.parameters(), lr=1e-3)
        train_losses, val_losses, metrics = [], [], []
        epoch_train_times, epoch_val_times = [], []  # 新增：記錄每個 epoch 的時間

        for epoch in range(num_epochs):
            self.train()
            total_loss = 0
            start_time = time.time()  # 訓練開始時間

            for images, targets in tqdm(train_loader, desc=f'Stage {stage} Epoch {epoch+1} [Train]'):
                images = images.to(self.device)
                if isinstance(targets, dict):  # Det
                    targets = {k: v.to(self.device) for k, v in targets.items()}
                    outputs = self(images)[0]
                elif isinstance(targets, torch.Tensor) and targets.ndim == 3:  # Seg
                    targets = targets.to(self.device)
                    outputs = self(images)[1]
                else:  # Class
                    targets = targets.to(self.device)
                    outputs = self(images)[2]

                optimizer.zero_grad()
                loss = criterion(outputs, targets)
                if prev_ewc:
                    loss += sum(ewc.penalty(self) for ewc in prev_ewc)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            train_time = time.time() - start_time  # 訓練時間
            epoch_train_times.append(train_time)
            avg_train_loss = total_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            self.eval()
            total_val_loss, metric_sum, num_batches = 0, 0, 0
            start_time = time.time()  # 驗證開始時間

            with torch.no_grad():
                for images, targets in tqdm(val_loader, desc=f'Stage {stage} Epoch {epoch+1} [Val]'):
                    images = images.to(self.device)
                    if isinstance(targets, dict):  # Det
                        targets = {k: v.to(self.device) for k, v in targets.items()}
                        outputs = self(images)[0]
                        metric = evaluate_map(self, val_loader) if evaluate_map else 0
                    elif isinstance(targets, torch.Tensor) and targets.ndim == 3:  # Seg
                        targets = targets.to(self.device)
                        outputs = self(images)[1]
                        preds = outputs.argmax(dim=1)
                        metric = self.compute_iou(preds, targets, num_classes=20)
                    else:  # Class
                        targets = targets.to(self.device)
                        outputs = self(images)[2]
                        preds = outputs.argmax(dim=1)
                        metric = (preds == targets).float().mean().item()

                    loss = criterion(outputs, targets)
                    total_val_loss += loss.item()
                    metric_sum += metric
                    num_batches += 1

            val_time = time.time() - start_time  # 驗證時間
            epoch_val_times.append(val_time)
            avg_val_loss = total_val_loss / len(val_loader)
            avg_metric = metric_sum / num_batches
            val_losses.append(avg_val_loss)
            metrics.append(avg_metric)

            metric_name = "mIoU" if stage == 1 else "mAP" if stage == 2 else "Top-1 Acc"
            print(f"[Stage {stage} Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, {metric_name}: {avg_metric:.6f}, Train Time: {format_time(train_time)}, Val Time: {format_time(val_time)}")

            torch.cuda.empty_cache()

        total_train_time = sum(epoch_train_times)
        total_val_time = sum(epoch_val_times)
        print(f"Stage {stage} Total Training Time: {format_time(total_train_time)}, Total Validation Time: {format_time(total_val_time)}")

        return train_losses, val_losses, metrics, avg_metric, epoch_train_times, epoch_val_times


# training way

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # 禁止用自動優化




class EWC(object):
    def __init__(self, model, dataloader, device, importance=1000):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.importance = importance
        self.params = {n: p.clone().detach() for n, p in model.named_parameters() if p.requires_grad}
        self.fisher = self.compute_fisher()

    def compute_fisher(self):
        fisher = {n: torch.zeros_like(p) for n, p in self.params.items()}
        self.model.eval()
        for images, targets in self.dataloader:
            images = images.to(self.device)
            if isinstance(targets, dict):  # det
                targets = {k: v.to(self.device) for k, v in targets.items()}
                outputs = self.model(images)[0]
                loss = self.detection_loss(outputs, targets).to(device)
            elif isinstance(targets, torch.Tensor) and targets.ndim == 3:  # seg
                targets = targets.to(self.device)
                outputs = self.model(images)[1]
                loss = nn.CrossEntropyLoss(ignore_index=21)(outputs, targets).to(device)
            else:  # class
                targets = targets.to(self.device)
                outputs = self.model(images)[2]
                loss = nn.CrossEntropyLoss()(outputs, targets).to(device)

            self.model.zero_grad()
            loss.backward()
            for n, p in self.model.named_parameters():
                if p.grad is not None:
                    fisher[n] += p.grad.data.pow(2) / len(self.dataloader)
        return fisher

    def penalty(self, model):
        loss = 0
        for n, p in model.named_parameters():
            if n in self.params:
                loss += (self.fisher[n] * (p - self.params[n]).pow(2)).sum()
        return self.importance * loss

    def detection_loss(self, outputs, targets):
        detection_loss_fn = DetectionLoss(n_anchors=3, c_det=10, stride=8).to(device)
        return detection_loss_fn(outputs, targets)



# 主程式

In [ ]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed=222)
    # -------------------------------
    # Detection Dataset (COCO)
    coco_train_dataset = COCODataset('data/mini-coco_det', split='train2017')
    coco_val_dataset   = COCODataset('data/mini-coco_det', split='val2017')
    det_train_loader = DataLoader(coco_train_dataset, batch_size=12, shuffle=True, collate_fn=collate_fn)
    det_val_loader   = DataLoader(coco_val_dataset, batch_size=12, shuffle=False, collate_fn=collate_fn)

    # -------------------------------
    # Segmentation Dataset (VOC)
    voc_train_dataset = VOCDataset('data/mini_voc_seg/train/images', 'data/mini_voc_seg/train/masks')
    voc_val_dataset   = VOCDataset('data/mini_voc_seg/val/images', 'data/mini_voc_seg/val/masks')
    seg_train_loader = DataLoader(voc_train_dataset, batch_size=12, shuffle=True)
    seg_val_loader   = DataLoader(voc_val_dataset, batch_size=12, shuffle=False)

    # -------------------------------
    # Classification Dataset (image160)
    image160_train_dataset = Image160Dataset('data/imagenette-160/train')
    image160_val_dataset   = Image160Dataset('data/imagenette-160/val')
    cls_train_loader = DataLoader(image160_train_dataset, batch_size=12, shuffle=True)
    cls_val_loader   = DataLoader(image160_val_dataset, batch_size=12, shuffle=False)

    # Loss functions
    seg_criterion = nn.CrossEntropyLoss(ignore_index=21).to(device)
    det_criterion = DetectionLoss(n_anchors=3, c_det=10, stride=16).to(device)
    cls_criterion = nn.CrossEntropyLoss().to(device)

    # Step 1: Train baselines
    # Segmentation baseline
    model_seg = MultiTaskModel(img_size=640)
    _, _, _, seg_miou_base, seg_train_times, seg_val_times = model_seg.train_stage(
        stage=1,
        train_loader=seg_train_loader,
        val_loader=seg_val_loader,
        criterion=seg_criterion,
        num_epochs=10,
    )

    # Detection baseline
    model_det = MultiTaskModel(img_size=640)
    coco_gt = COCO('data/mini-coco_det/annotations/instances_val2017.json')
    _, _, _, det_map_base, det_train_times, det_val_times = model_det.train_stage(
        stage=2,
        train_loader=det_train_loader,
        val_loader=det_val_loader,
        criterion=det_criterion,
        evaluate_map=lambda model, val_loader: evaluate_map(model, val_loader, coco_gt),
        num_epochs=10,
    )

    # Classification baseline
    model_cls = MultiTaskModel(img_size=640)
    _, _, _, cls_acc_base, cls_train_times, cls_val_times = model_cls.train_stage(
        stage=3,
        train_loader=cls_train_loader,
        val_loader=cls_val_loader,
        criterion=cls_criterion,
        num_epochs=10,
    )

    # Step 2: Sequential training with EWC
    model = MultiTaskModel(img_size=640)
    ewc_list = []

    # Stage 1: Segmentation
    print("Stage 1: Seg")
    seg_losses, seg_val_losses, seg_mious, _, ewc_seg_train_times, ewc_seg_val_times = model.train_stage(
        stage=1,
        train_loader=seg_train_loader,
        val_loader=seg_val_loader,
        criterion=seg_criterion,
        num_epochs=10,
    )
    ewc_seg = EWC(model, seg_val_loader, device, importance=1000)
    ewc_list.append(ewc_seg)

    # Stage 2: Detection
    print("Stage 2: Det")
    coco_gt = COCO('data/mini-coco_det/annotations/instances_val2017.json')
    det_losses, det_val_losses, det_maps, _, ewc_det_train_times, ewc_det_val_times = model.train_stage(
        stage=2,
        train_loader=det_train_loader,
        val_loader=det_val_loader,
        criterion=det_criterion,
        evaluate_map=lambda model, val_loader: evaluate_map(model, val_loader, coco_gt),
        num_epochs=10,
        prev_ewc=ewc_list,
    )
    ewc_det = EWC(model, det_val_loader, device, importance=1000)
    ewc_list.append(ewc_det)

    # Stage 3: Class
    print("Stage 3: Class")
    cls_losses, cls_val_losses, cls_accs, _, ewc_cls_train_times, ewc_cls_val_times = model.train_stage(
        stage=3,
        train_loader=cls_train_loader,
        val_loader=cls_val_loader,
        criterion=cls_criterion,
        num_epochs=10,
        prev_ewc=ewc_list,
    )

    # Step 3: Evaluate forgetting
    model.eval()
    eval_start_time = time.time()  # 總評估開始時間

    with torch.no_grad():
        # Segmentation
        seg_miou = 0
        seg_eval_start = time.time()
        for images, targets in seg_val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)[1]
            preds = outputs.argmax(dim=1)
            seg_miou += model.compute_iou(preds, targets, num_classes=20)
        seg_miou /= len(seg_val_loader)
        seg_eval_time = time.time() - seg_eval_start

        # Detection
        det_eval_start = time.time()
        det_map = evaluate_map(model, det_val_loader, coco_gt)
        det_eval_time = time.time() - det_eval_start

        # Classification
        cls_acc = 0
        cls_eval_start = time.time()
        for images, targets in cls_val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)[2]
            preds = outputs.argmax(dim=1)
            cls_acc += (preds == targets).float().mean().item()
        cls_acc /= len(cls_val_loader)
        cls_eval_time = time.time() - cls_eval_start

    total_eval_time = time.time() - eval_start_time

    # Performance drops
    seg_drop = (seg_miou_base - seg_miou) * 100
    det_drop = (det_map_base - det_map) * 100
    cls_drop = (cls_acc_base - cls_acc) * 100

    # Print results
    print(f"\nFinal Evaluation:")
    print(f"Segmentation mIoU: {seg_miou:.6f} (Baseline: {seg_miou_base:.6f}, Drop: {seg_drop:.4f}%)")
    print(f"Detection mAP: {det_map:.6f} (Baseline: {det_map_base:.6f}, Drop: {det_drop:.4f}%)")
    print(f"Classification Top-1 Acc: {cls_acc:.6f} (Baseline: {cls_acc_base:.6f}, Drop: {cls_drop:.4f}%)")

    # Verify drops
    if (seg_drop <= 5.0) and (det_drop <= 5.0) and (cls_drop <= 5.0):
        print(f"Segmentation mIoU drop {seg_drop:.4f}% <= 5%")
        print(f"Detection mAP drop {det_drop:.4f}% <= 5%")
        print(f"Classification Top-1 Acc drop {cls_drop:.4f}% <= 5%")
        print("All performance drops are within 5% of baselines.")
    if seg_drop > 5.0:
        print(f"Segmentation mIoU drop {seg_drop:.4f}% > 5%")
    if det_drop > 5.0:
        print(f"Detection mAP drop {det_drop:.4f}% > 5%")
    if cls_drop > 5.0:
        print(f"Classification Top-1 Acc drop {cls_drop:.4f}% > 5%")

    print(f"MultiTask Model Parameters: {count_parameters(model):,} (Million: {count_parameters(model)/1e6:.2f}M)")
    # Print timing and parameter summary
    print("\nTraining and Evaluation Time Summary:")
    print(f"Segmentation Baseline - Total Train Time: {format_time(sum(seg_train_times))}, Total Val Time: {format_time(sum(seg_val_times))}")
    print(f"Detection Baseline - Total Train Time: {format_time(sum(det_train_times))}, Total Val Time: {format_time(sum(det_val_times))}")
    print(f"Classification Baseline - Total Train Time: {format_time(sum(cls_train_times))}, Total Val Time: {format_time(sum(cls_val_times))}")
    print(f"EWC Stage 1 (Seg) - Total Train Time: {format_time(sum(ewc_seg_train_times))}, Total Val Time: {format_time(sum(ewc_seg_val_times))}")
    print(f"EWC Stage 2 (Det) - Total Train Time: {format_time(sum(ewc_det_train_times))}, Total Val Time: {format_time(sum(ewc_det_val_times))}")
    print(f"EWC Stage 3 (Cls) - Total Train Time: {format_time(sum(ewc_cls_train_times))}, Total Val Time: {format_time(sum(ewc_cls_val_times))}")
    print(f"Evaluation Times - Seg: {format_time(seg_eval_time)}, Det: {format_time(det_eval_time)}, Cls: {format_time(cls_eval_time)}, Total: {format_time(total_eval_time)}")
if __name__ == "__main__":
    main()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
建立顏色對應表...
找到 22 種顏色，映射為 0~19 有效類別 + 背景類
顏色對應表:
  RGB(0, 0, 0) -> 類別21
  RGB(255, 255, 255) -> 類別21
  RGB(np.uint8(0), np.uint8(0), np.uint8(128)) -> 類別0
  RGB(np.uint8(0), np.uint8(64), np.uint8(0)) -> 類別1
  RGB(np.uint8(0), np.uint8(64), np.uint8(128)) -> 類別2
  RGB(np.uint8(0), np.uint8(128), np.uint8(0)) -> 類別3
  RGB(np.uint8(0), np.uint8(128), np.uint8(128)) -> 類別4
  RGB(np.uint8(0), np.uint8(192), np.uint8(0)) -> 類別5
  RGB(np.uint8(64), np.uint8(0), np.uint8(0)) -> 類別6
  RGB(np.uint8(64), np.uint8(0), np.uint8(128)) -> 類別7
  RGB(np.uint8(64), np.uint8(128), np.uint8(0)) -> 類別8
  RGB(np.uint8(64), np.uint8(128), np.uint8(128)) -> 類別9
  RGB(np.uint8(128), np.uint8(0), np.uint8(0)) -> 類別10
  RGB(np.uint8(128), np.uint8(0), np.uint8(128)) -> 類別11
  RGB(np.uint8(128), np.uint8(64), np.uint8(0)) -> 類別12
  RGB(np.uint8(128),

Stage 1 Epoch 1 [Val]: 100%|██████████| 5/5 [00:14<00:00,  2.97s/it]


[Stage 1 Epoch 1] Train Loss: 2.2665, Val Loss: 1.6914, mIoU: 0.161316, Train Time: 00:01:03, Val Time: 00:00:14


Stage 1 Epoch 2 [Val]: 100%|██████████| 5/5 [00:14<00:00,  2.98s/it]


[Stage 1 Epoch 2] Train Loss: 1.5075, Val Loss: 1.0837, mIoU: 0.176216, Train Time: 00:01:04, Val Time: 00:00:14


...

[Stage 3 Epoch 9] Train Loss: 1.0146, Val Loss: 2.4481, Top-1 Acc: 0.383333, Train Time: 00:00:05, Val Time: 00:00:01


Stage 3 Epoch 10 [Val]: 100%|██████████| 5/5 [00:01<00:00,  4.21it/s]


[Stage 3 Epoch 10] Train Loss: 1.0186, Val Loss: 3.0156, Top-1 Acc: 0.366667, Train Time: 00:00:05, Val Time: 00:00:01
Stage 3 Total Training Time: 00:00:55, Total Validation Time: 00:00:12


Evaluating: 100%|██████████| 5/5 [00:01<00:00,  3.06it/s]


Total detections: 7829
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.65s).
Accumulating evaluation results...
DONE (t=0.12s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | a